<a href="https://colab.research.google.com/github/writezubair-cpu/ZAK-TEST_AI_Training/blob/main/Copy_of_crewai_multiagent_level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gitex workshop: Create Multi Agents to Research and Write an Article

In this workshop, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

https://huggingface.co/spaces/decodingdatascience/fromprompttopost

In [ ]:
#install packages
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
#setting the model
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.



## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
# Retrieve API key from Colab secrets and set it in environment variables
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [ ]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [ ]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "d40bd0a342206683691c064cabada9c14f90c3f5" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."
              "Also provide history or start date",
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices."
              "You also check for grammar, spelling, and overall coherence. "
              "You ensure the tone is consistent and the content is engaging and accurate.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.2500 words",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=4
)

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

## Running the Crew
## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "GPT 5.2"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on GPT 5.2.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
To create a comprehensive content plan on GPT 5.2, I need to gather the latest information about GPT 5.2, identify the target audience, and develop a detailed content outline. I'll begin by searching the internet for the latest trends, key players, and noteworthy news on GPT 5.2.

Action: Search the internet
Action Input: {"search_query": "GPT 5.2 latest trends news 2023"} 


Search results: Title: Introducing GPT-5.2 - OpenAI
Link: https://openai.com/index/introducing-gpt-5-2/
Snippet: Overall, GPT‑5.2 brings significant improvements in general intelligence, long-conte

In [ ]:
from IPython.display import Markdown
Markdown(result)

my best complete final answer to the task.  

```markdown
# GPT 5.2: Ushering in a New Era of AI Advancements

Artificial intelligence has seen monumental strides over the past few years, with OpenAI consistently leading the charge. With the release of GPT 5.2, the landscape of AI technology is poised for another significant leap forward. In this opinion piece, we'll delve into the improvements made in GPT 5.2, explore its professional applications, and assess its broader impact on various industries. As we navigate through these advancements, we'll also touch upon the potential challenges and ethical considerations that accompany this new era of AI.

## Enhanced Capabilities of GPT 5.2

GPT 5.2 emerges as a refined version of its predecessors, boasting notable improvements in natural language understanding and generation. The model has been meticulously trained on a diverse dataset, enhancing its ability to comprehend context with greater accuracy. This evolution is not merely incremental; it represents a paradigm shift in how AI models can interpret and generate human-like text.

One of the key enhancements in GPT 5.2 is its ability to process nuanced language subtleties, including idioms and cultural references, which were previously challenging for AI models. This improvement stems from an advanced neural network architecture that allows the model to draw connections between disparate pieces of information more effectively. Consequently, GPT 5.2 can produce text that is not only coherent but also contextually relevant, making it an invaluable tool for industries reliant on precise communication.

## Professional Applications: Transforming Workflows

The professional applications of GPT 5.2 are vast, transcending traditional boundaries and redefining workflows across sectors. In the field of content creation, for example, GPT 5.2 offers unparalleled assistance by generating high-quality drafts, providing writers with a robust starting point. This capability not only accelerates the writing process but also allows creators to focus on refining their ideas rather than getting bogged down in the initial drafting phase.

Moreover, GPT 5.2 is poised to revolutionize customer service by providing instant, intelligent responses to customer queries. Its improved natural language understanding enables it to handle complex questions with ease, reducing the need for human intervention and thus streamlining operations. In healthcare, the model's ability to process and analyze vast amounts of data can aid in diagnostics and personalized treatment plans, showcasing its potential to contribute significantly to patient care.

## Industry Impact: A Catalyst for Change

The release of GPT 5.2 is expected to have far-reaching implications across multiple industries. In the financial sector, for instance, the model's advanced data processing capabilities can enhance predictive analytics, leading to more informed decision-making. Financial institutions could leverage GPT 5.2 to analyze market trends and consumer behavior, thereby gaining a competitive edge.

In the realm of education, GPT 5.2 holds the promise of personalized learning experiences. By tailoring educational content to individual learning styles and paces, the model can foster a more inclusive and effective educational environment. Furthermore, its ability to facilitate language translation and interpretation stands to break down communication barriers, promoting cross-cultural understanding and collaboration.

## Ethical Considerations and Challenges

Despite its impressive advancements, GPT 5.2 is not without its challenges. The ethical implications of deploying such powerful AI models are significant and warrant careful consideration. One of the primary concerns is the potential for misuse, such as generating deepfakes or spreading misinformation. As with any technological advancement, the onus is on developers and users to implement safeguards that prevent such occurrences.

Additionally, the deployment of GPT 5.2 raises questions about job displacement, as the automation of tasks traditionally performed by humans becomes more prevalent. It is crucial for industries to anticipate these changes and proactively invest in upskilling and reskilling initiatives to ensure a smooth transition for the workforce.

## Conclusion: Embracing the Future with Caution

GPT 5.2 represents a monumental step forward in the field of artificial intelligence. Its enhanced capabilities, professional applications, and potential industry impact underscore its significance as a transformative tool. However, as we embrace these advancements, it is imperative to remain vigilant about the ethical considerations and challenges they present.

In my opinion, the key to harnessing the full potential of GPT 5.2 lies in striking a balance between innovation and responsibility. By doing so, we can unlock new opportunities, drive progress, and ultimately create a future where AI serves as a powerful ally in addressing some of the world's most pressing challenges.

As we stand on the cusp of this new era, the dialogue surrounding AI development must continue to evolve, ensuring that technologies like GPT 5.2 are utilized in ways that are both beneficial and ethical. The future of AI is undoubtedly bright, and with thoughtful stewardship, it can illuminate the path to a more prosperous and equitable world.
```

# Deploy this crew in gradio

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic and history of it atleast release date if it is for Model.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 2500 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="AI Agent: Prompt to Post",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2bec42efb38ede109a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
